## **Predict Future Sales: 全部ぶっこんでXGBoost**
Final project for "How to win a data science competition" Coursera course<br>
https://www.kaggle.com/c/competitive-data-science-predict-future-sales/overview

#### **Data Preparation**

In [2]:
import pandas as pd 
import numpy as np
import glob 
import os 
import matplotlib.pyplot as plt

In [3]:
rawdata = {}
rawdata["item_cat"] = pd.read_csv(os.path.join("rawdata","item_categories.csv"),encoding="utf-8",index_col=1)
rawdata["items"] = pd.read_csv(os.path.join("rawdata","items.csv"),encoding="utf-8",index_col=1)
rawdata["sales_train"] = pd.read_csv(os.path.join("data","sales_train.csv"),encoding="utf-8",parse_dates=[0],index_col=0)
rawdata["shops"] = pd.read_csv(os.path.join("rawdata","shops.csv"),encoding="utf-8")
rawdata["test"] = pd.read_csv(os.path.join("rawdata","test.csv"),encoding="utf-8")

In [7]:
rawdata["shops"]

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
5,"Вологда ТРЦ ""Мармелад""",5
6,"Воронеж (Плехановская, 13)",6
7,"Воронеж ТРЦ ""Максимир""",7
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9


In [3]:
for key in rawdata.keys():
    df = rawdata[key]
    print(df.head(3))
    print("[Nan Values]")
    print(df.isna().sum())
    print()

                       item_category_name
item_category_id                         
0                 PC - Гарнитуры/Наушники
1                        Аксессуары - PS2
2                        Аксессуары - PS3
[Nan Values]
item_category_name    0
dtype: int64

                                                 item_name  item_category_id
item_id                                                                     
0                ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D                40
1        !ABBYY FineReader 12 Professional Edition Full...                76
2            ***В ЛУЧАХ СЛАВЫ   (UNV)                    D                40
[Nan Values]
item_name           0
item_category_id    0
dtype: int64

            date_block_num  shop_id  item_id  item_price  item_cnt_day
date                                                                  
2013-02-01               0       59    22154       999.0           1.0
2013-03-01               0       25     2552       899.0           1.0

#### **簡易的な特徴量作成**
shop_id,item_id,年の情報を入れる. 目的変数は月の合計値とする

In [4]:
rawdata["train"] = rawdata["sales_train"].groupby(by=["date_block_num","shop_id","item_id"]).sum()[["item_cnt_day"]].reset_index()
rawdata["price"] = rawdata["sales_train"].groupby(by=["item_id"]).mean()[["item_price"]].reset_index()

rawdata["train"] = pd.merge(rawdata["train"], rawdata["price"], on="item_id", how="inner")

rawdata["test"] = pd.merge(rawdata["test"], rawdata["price"], on="item_id", how="outer")
rawdata["test"]["date_block_num"] = 34

#rawdata["train"] = rawdata["train"].set_index("ID",drop=True)
rawdata["test"] = rawdata["test"].set_index("ID",drop=True)

In [7]:
rawdata["train"]

,date_block_num,shop_id,item_id,item_cnt_day,item_price
0,0,0,32,6.0,249.62924
1,0,1,32,7.0,249.62924
2,0,3,32,3.0,249.62924
3,0,4,32,2.0,249.62924
4,0,6,32,8.0,249.62924
...,...,...,...,...,...
1609119,33,55,12733,2.0,1490.00000
1609120,33,55,13092,1.0,2000.00000
1609121,33,55,16797,1.0,790.00000
1609122,33,55,18060,1.0,172.00000


#### **XGBRegressorによる回帰**

In [8]:
import xgboost as xgb

target = "item_cnt_day"
features = ["date_block_num","shop_id","item_id"]

trainX = rawdata["train"][features].values
trainY = rawdata["train"][target].values

reg = xgb.XGBRegressor(tree_method='gpu_hist')
reg.fit(trainX,trainY)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='gpu_hist', validate_parameters=False, verbosity=None)

In [9]:
#predict
testX = rawdata["test"][features].values
rawdata["test"]["predict"] = reg.predict(testX)

# submission
sub = pd.read_csv(os.path.join("submission","sample_submission.csv"),index_col=0)
my_sub = sub.join(rawdata["test"][["predict"]])
my_sub["item_cnt_month"] = my_sub["predict"]
my_sub = my_sub.drop(columns={"predict"})

my_sub.to_csv(os.path.join("submission","XGBoost.csv"))

In [ ]:
monthly_sales = monthly_sales.reset_index()

merged_table = pd.merge(monthly_sales,shop_infos_en,on="shop_id")